# Part 7 Friend Recommendation

In [4]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder\
    .config("spark.driver.memory", "32g")\
    .config("spark.executor.memory","32g")\
    .config("spark.driver.maxResultSize","0")\
    .config("spark.sql.autoBroadcastJoinThreshold","-1")\
    .config("spark.sql.broadcastTimeout","1200")\
    .config("spark.default.parallelism", "32")\
    .config("spark.executor.heartbeatInterval", "3600")\
    .config("spark.local.dir","D:\\Data")\
    .appName("part7").getOrCreate()

In [48]:
review = spark.read.csv("part4_topicmodeling_review.csv", header=True, multiLine=True)
business = spark.read.csv("part1_dataclean_business.csv",header=True, multiLine=True)
user = spark.read.csv("part1_dataclean_user.csv", header=True, multiLine=True)
user_friends = user.select("user_id", "friends")

In [50]:
def get_business_details(business_list):
    return business.filter(col("business_id").isin(business_list)).orderBy("stars", ascending=False)

def get_friends_recommendation(user_id, limit=10):
    friends = user_friends.filter(col("user_id")==user_id).limit(1)
    if friends.count()<1:
        return None
    friend_str = friends.select("friends").rdd.collect()[0][0].replace(" ", "")
    if friend_str=="":
        return None
    friend_list = list(friend_str.split(","))
    friends_businesses = review.filter(
         (col("superscore")>=3.0) &(col("user_id").isin(friend_list)))\
        .orderBy("superscore", ascending=False)\
        .select("business_id").distinct().limit(limit).rdd.collect()
    if len(friends_businesses)==0:
        return None
    business_list = []
    for row in friends_businesses:
        business_list.append(row.business_id)
    return get_business_details(business_list)
get_friends_recommendation("uLhdaZUsVvT0gbNTdOSzDg").toPandas()

,business_id,name,city,state,stars,review_count,categories,latitude,longitude,is_open,postal_code
0,n73rxa6e6-fTIxQzfv4BuA,Mother’s Bistro & Bar,Portland,OR,4.5,4466,"American (Traditional), Restaurants, Bars, Nig...",45.522196,-122.673868,1,97204
1,1JKK_kmQ6Kxc6NzhQhcAOA,Sivalai Thai Restaurant,Portland,OR,4.5,833,"Restaurants, Thai",45.5190945,-122.6131884,1,97215
2,9P-lp3AWDXGayDqJz9VPwQ,Marukin Ramen,Portland,OR,4.0,616,"Restaurants, Ramen, Noodles, Japanese",45.522407,-122.65937,1,97214
3,z1QhExIrJH0cnBdQKrsk0g,Piazza Italia,Portland,OR,4.0,973,"Restaurants, Shopping, Salad, Delis, Art Galle...",45.5287804,-122.6829301,1,97209
4,nEaWb1R9wFRI6sTWBkymAA,Kizuki Ramen & Izakaya,Portland,OR,4.0,955,"Ramen, Japanese, Restaurants, Izakaya, Noodles",45.5217636,-122.8021536,1,97229
5,jXnMy7-VQ7eqRBtjXAacuQ,Lucky Devil Lounge,Portland,OR,4.0,112,"Restaurants, Bars, Adult Entertainment, Nightl...",45.5017514,-122.6590969,1,97202
6,4VT8kIUlTa4d3mwWrGJ5Zg,Pepe le Moko,Portland,OR,4.0,439,"Cocktail Bars, Nightlife, Bars, Restaurants, S...",45.5218730302,-122.6811734763,1,97205
7,d69y3CN9_SQKrEnXXqQr8g,King Burrito Mexican Food,Portland,OR,4.0,534,"Mexican, Restaurants, Breakfast & Brunch, Burgers",45.5769598,-122.6970866,1,97217
8,nb6XRTOkX2mtWianzv819Q,Shigezo Izakaya,Portland,OR,3.5,678,"Tapas/Small Plates, Japanese, Ramen, Restauran...",45.5176574,-122.6825351,1,97205
9,qgq0usjnOA8qTWHdxbuzZA,Eclectic Kitchen,Portland,OR,3.5,137,"Food, Salad, Sandwiches, Restaurants, Bakeries...",45.548143,-122.6118289,1,97213


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "E:\IDE\Conda3\envs\CS5344V3\lib\site-packages\py4j\java_gateway.py", line 1205, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "E:\IDE\Conda3\envs\CS5344V3\lib\socket.py", line 589, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "E:\IDE\Conda3\envs\CS5344V3\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "E:\IDE\Conda3\envs\CS5344V3\lib\site-packages\py4j\java_gateway.py", line 1217, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 58665)
Traceback 